In [1]:
import numpy as np
from pyrep import PyRep
from arm import CtRobot
from my_rigid_kinematics import dh_robot_config
import sympy as sp
import os
import transforms3d as t3d
from pyrep.backend import vrep

pi = np.pi

In [2]:
pr = PyRep()
pr.launch(os.getcwd() + '/Modified_DH.ttt', headless=True)

In [3]:
pr.start()
ct_robot = CtRobot()

## My calibration (based on vrep 'reference_frame')

In [4]:
# Grab modified DH parameters from V-REP using the dummy 'DH_frame'
# Need to set up their orientation following modified DH convention
D = []
theta = []
alpha = []
a = []
L = []
jointType = ['p', 'p', 'r', 'r', 'r', 'r', 'p']
for i in range(ct_robot._num_joints):
    j = i+1
    tmp_alpha = ct_robot.DH_frames[i+1].get_orientation(relative_to=ct_robot.DH_frames[i])
    tmp_a = ct_robot.DH_frames[i+1].get_position(relative_to=ct_robot.DH_frames[i])
    if tmp_alpha[0] < -1e-5 or tmp_alpha[0] > 1e-5:
        alpha.append(tmp_alpha[0])
    else:
        alpha.append(0)
    a.append(tmp_a[0])
    
    tmp_d = -ct_robot.DH_frames[j-1].get_position(relative_to=ct_robot.DH_frames[j])[2]
    tmp_theta = -ct_robot.DH_frames[j-1].get_orientation(relative_to=ct_robot.DH_frames[j])[2]
    if tmp_theta < -1e-5 or tmp_theta > 1e-5:
        theta.append(tmp_theta)
    else:
        theta.append(0)   
    D.append(tmp_d)
# Center of mass of link with respect to each joint    
for i in range(ct_robot._num_joints+1):
    L.append(ct_robot.COMs[i].get_position(relative_to=ct_robot.DH_frames[i]))
    
Tbase = np.concatenate((np.array(ct_robot.DH_frames[0].get_matrix()).reshape(3,4), np.array([[0,0,0,1]])))

num_joints = ct_robot._num_joints

In [ ]:
# Grab modified DH parameters from V-REP using the dummy 'DH_frame'
# Need to set up their orientation following modified DH convention
D = []
theta = []
alpha = []
a = []
L = []
jointType = ['p', 'p', 'r', 'r', 'r', 'r', 'p']
for i in range(ct_robot._num_joints):
    tmp_alpha = ct_robot.DH_frames[i+1].get_orientation(relative_to=ct_robot.DH_frames[i])[0]
    tmp_a = ct_robot.DH_frames[i+1].get_position(relative_to=ct_robot.DH_frames[i])[0]
    if tmp_alpha < -1e-5 or tmp_alpha > 1e-5:
        alpha.append(tmp_alpha)
    else:
        alpha.append(0)
    a.append(tmp_a)
    
    tmp_d = ct_robot.DH_frames[i+1].get_position(relative_to=ct_robot.DH_frames[i])[2]
    tmp_theta = ct_robot.DH_frames[i+1].get_orientation(relative_to=ct_robot.DH_frames[i])[2]
    if tmp_theta < -1e-5 or tmp_theta > 1e-5:
        theta.append(tmp_theta)
    else:
        theta.append(0)   
    D.append(tmp_d)
# Center of mass of link with respect to each joint    
for i in range(ct_robot._num_joints+1):
    L.append(ct_robot.COMs[i].get_position(relative_to=ct_robot.DH_frames[i]))
    
Tbase = np.concatenate((np.array(ct_robot.DH_frames[0].get_matrix()).reshape(3,4), np.array([[0,0,0,1]])))
num_joints = ct_robot._num_joints

In [5]:
robot = dh_robot_config(num_joints, alpha, theta, D, a, jointType, Tbase)

In [6]:
robot.initKinematicTransforms()

## V-REP

In [18]:
for i in range(7):
    print('---------------------------')
    print('Joint {}'.format(i+1))
    print('D: {}, theta: {}, a[i-1]: {}, alpha[i-1]: {}'.format(D[i],theta[i],a[i],alpha[i]))
#print("D {}, theta {}\n"
    
    #D, '\n',  '\n',theta, '\n',  '\n',a, '\n', '\n', alpha)

---------------------------
Joint 1
D: -0.0, theta: 0, a[i-1]: 0.0, alpha[i-1]: 0
---------------------------
Joint 2
D: 2.2900366047906573e-09, theta: 0, a[i-1]: 0.05849161744117737, alpha[i-1]: 1.570797085762024
---------------------------
Joint 3
D: 1.7881393432617188e-07, theta: 0, a[i-1]: 0.1404283195734024, alpha[i-1]: -1.5707967281341553
---------------------------
Joint 4
D: -5.416572093963623e-06, theta: 0, a[i-1]: 3.4868717193603516e-06, alpha[i-1]: 1.5708014965057373
---------------------------
Joint 5
D: 1.6242265701293945e-06, theta: 1.5708030462265015, a[i-1]: 1.1622905731201172e-06, alpha[i-1]: 0
---------------------------
Joint 6
D: 2.3581287678098306e-06, theta: -1.5707963705062866, a[i-1]: 0.08000022172927856, alpha[i-1]: 1.5707966089248657
---------------------------
Joint 7
D: 0.03896486759185791, theta: 0, a[i-1]: 2.562999725341797e-06, alpha[i-1]: -1.5707961320877075


In [ ]:
theta = 0
alpha = alpha[3]
D = D[3]
a =a[3]
A = np.array([
    [np.cos(theta), -np.sin(theta), 0, a],
    [np.sin(theta)*np.cos(alpha), np.cos(theta)*np.cos(alpha), -np.sin(alpha), -D*np.sin(alpha)],
    [np.sin(theta)*np.sin(alpha), np.cos(theta)*np.sin(alpha), np.cos(alpha), D*np.cos(alpha)],
    [0, 0, 0, 1]])

print(A)

In [ ]:
for i in range(7):
    T = np.eye(4)
    T[:3, :] = np.array(ct_robot.DH_frames[i+1].get_matrix()). \
    reshape(3,4)
    print(T)

In [ ]:
for i in range(7):
    T = np.eye(4)
    T[:3, :] = np.array(ct_robot.DH_frames[i+1].get_matrix()).reshape(3,4)
    Tp = sp.lambdify(robot.q, robot._Tjoint[i])(*tuple(joint_pos))
    print('diff:', np.linalg.norm(Tp - T))
    if np.linalg.norm(Tp - T) > 1e-4:
        print(i)
        print(T)
        print(Tp)

## DH-FK

In [14]:
# set_jot_pos = [0.01, 0.02, 0.4, 0.5, 0, 0.3, 0.05]
set_jot_pos = [0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0]
ct_robot.set_joint_positions(set_jot_pos)

In [15]:
joint_pos = [ct_robot.joints[i].get_joint_position() for i in range(ct_robot._num_joints)]
print('Joint position is', joint_pos)
print('Tip position is', ct_robot._ik_tip.get_position())
print('Tip orientation is', ct_robot._ik_tip.get_orientation())

Joint position is [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Tip position is [0.001053992542438209, -0.7582254409790039, 0.19892005622386932]
Tip orientation is [1.5708180665969849, 4.291534423828125e-06, -1.5707895755767822]


In [17]:
t3d.euler.mat2euler(np.array([[1,0,0],[0,0,1],[0,-1,0]]))

(-1.5707963267948966, -0.0, 0.0)

In [16]:
for i in range(7):
    
#     #vrep
#     T = np.eye(4)
# #     # Filtering 
# #     ori = np.array(ct_robot.DH_frames[i+1].get_orientation(ct_robot.DH_frames[i]))
# #     for j in range(3):
# #         if ori[j] > -1e-6 and ori[j] < 1e-6:
# #             ori[j] == 0
# #     Rt = t3d.euler.euler2mat(*tuple(ori.tolist()), 'rxyz')
# #     T[:3, :3] = Rt
# #     T[:3, 3] = ct_robot.DH_frames[i+1].get_position(ct_robot.DH_frames[i])


    #vrep
    T = np.eye(4)
    T[:3, :] = np.array(ct_robot.DH_frames[i+1].get_matrix(ct_robot.DH_frames[i])). \
    reshape(3,4)
    
    # FK
    Tp = sp.lambdify(robot.q, robot._Tj2j[i])(*tuple(joint_pos))
    print('diff:', np.linalg.norm(Tp - T))
    if np.linalg.norm(Tp - T) > 1e-4:
        print('Wrong match of joint %d'%(i+1))
        print('getting from V-REP')
        print(T)
        print('----------------------------------')
        print('getting from FK')
        print(Tp)
        print('----------------------------------')

diff: 6.756653009309692e-07
diff: 3.3787713722734226e-07
diff: 2.3754775355250753e-07
diff: 2.1287110252758947e-05
diff: 2.0016052834099183
Wrong match of joint 5
getting from V-REP
[[-6.79492950e-06  5.90085983e-06  1.00000000e+00  1.16229057e-06]
 [ 1.00000000e+00  1.19209290e-07  6.73532486e-06  7.99999237e-02]
 [-1.78813934e-07  1.00000000e+00 -5.96046448e-06 -2.31764807e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
----------------------------------
getting from FK
[[-6.71943160e-06 -1.00000000e+00  0.00000000e+00  1.16229057e-06]
 [ 1.00000000e+00 -6.71943160e-06  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.62422657e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
----------------------------------
diff: 1.8885224889470909e-06
diff: 0.001063072653392345
Wrong match of joint 7
getting from V-REP
[[ 1.00000000e+00 -1.26522210e-07  1.26358898e-07  2.56299973e-06]
 [-1.26358870e-07  1.78813934

In [ ]:
T_j7 = sp.lambdify(robot.q, robot._Tjoint[6])(*tuple(joint_pos))
pos_j7 = T_j7[:3, 3]
R_j7 = T_j7[:3, :3]
ori_j7 = t3d.euler.mat2euler(R_j7, 'sxyz')
print('Tip position is', pos_j7)
print('Tip orientation is', ori_j7)

## Compare

In [ ]:
for i in range(ct_robot._num_joints):
    

In [ ]:
pr.shutdown()
